To apply the artificial immune pattern recognition to perform a task of structure damage Classification

The AIRS algorithm stands for Artificial Immune Recognition System. It is a machine learning algorithm inspired by the biological immune system,
particularly its adaptive nature in recognizing and responding to pathogens.
Key Concepts:
Memory cells: Represent learned patterns (analogous to memory B-cells in biology)

Stimulation: Measures similarity between an input (antigen) and memory cells (antibodies)

Cloning and mutation: New candidate memory cells are generated through cloning and mutation of stimulated cells

Resource allocation: Controls the growth of memory cells to avoid overfitting

Affinity maturation: Iterative improvement of memory cells for better classification accuracy

AIRS Workflow Summary:
Input an instance (antigen).

Compare it to memory cells to calculate affinity.

Select best matching cells, then clone and mutate them.

Evaluate and update the memory cell pool.

Repeat until classification performance stabilizes.

Applications:
Pattern recognition

Medical diagnosis

In [4]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# --- Parameters ---
CLONE_RATE = 5
MUTATION_RATE = 0.1
MEMORY_POOL_SIZE = 50
AFFINITY_THRESHOLD = 0.9
MAX_EPOCHS = 100

# --- Affinity (similarity) ---
def affinity(a, b):
    return np.exp(-np.linalg.norm(a - b))  # Gaussian similarity

# --- Mutation ---
def mutate(cell, rate=0.1):
    return cell + np.random.normal(0, rate, size=cell.shape)

# --- AIRS Classifier ---
class AIRSClassifier:
    def __init__(self):
        self.memory_cells = []
        self.memory_labels = []

    def train(self, X, y):
        self.memory_cells = []
        self.memory_labels = []

        # Initialize memory with random samples
        indices = np.random.choice(len(X), MEMORY_POOL_SIZE, replace=False)
        self.memory_cells = X[indices]
        self.memory_labels = y[indices]

        for epoch in range(MAX_EPOCHS):
            for i, x in enumerate(X):
                label = y[i]

                # Calculate affinities
                affinities = np.array([affinity(x, m) for m in self.memory_cells])
                best_idx = np.argmax(affinities)

                # Clone and mutate
                if affinities[best_idx] < AFFINITY_THRESHOLD:
                    clones = [mutate(self.memory_cells[best_idx], MUTATION_RATE) for _ in range(CLONE_RATE)]
                    clone_affinities = [affinity(x, c) for c in clones]
                    best_clone = clones[np.argmax(clone_affinities)]

                    # Replace memory cell with best clone
                    self.memory_cells[best_idx] = best_clone
                    self.memory_labels[best_idx] = label

    def predict(self, X):
        preds = []
        for x in X:
            affinities = np.array([affinity(x, m) for m in self.memory_cells])
            best_idx = np.argmax(affinities)
            preds.append(self.memory_labels[best_idx])
        return np.array(preds)


In [5]:
# Load your dataset (replace with actual sensor/feature data)
from sklearn.datasets import load_iris  # Simulating structural data
data = load_iris()
X = data.data
y = data.target

# Normalize features
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train and test AIRS
clf = AIRSClassifier()
clf.train(X_train, y_train)

y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.9111111111111111
